## Loading .properties file

In [1]:
# Reading .properties file, returns a dictionary
def loadProperties(filepath, sep='=', com='#'):
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(com):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = key_value[1].strip()
                props[key] = value
#                 try:
#                     props[value].append(key)
#                 except:
#                     props[value] = [key]
    return props

In [2]:
props = loadProperties('submitActionClass.properties')

# Thats how props looks now
# print('6b : ', props['6b'])
# print('1a1 : ', props['1a1'])
# print('6a2 : ', props['6a2'])
i = 0
for key in props:
    print(key, " - ", props[key])
    i += 1
    if i > 5:
        break

ningaloo  -  1a
exmouth  -  1a1
tantabiddi  -  1a1
coral\u0020bay  -  1a2
christmas  -  ChristmasIsland
indonesia  -  Indonesia


## Loading WEKA File

In [3]:
import csv

# Reading WEKA file, returning a dictionary
def loadWEKA(filename, limit=0):
    attributes = [] # Will store csv columns names here
    data = []       # Will store readed file here
    
    with open(filename, "r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',', quotechar="'")
        
        line_num = 0
        dataBegan = False
        
        for row in reader:
            # Getting info from @ATTRIBUTE's
            if not dataBegan:
                if len(row) > 0:
                    # Extracting @ATTRIBUTE's
                    row_splitted = row[0].split()
                    if row_splitted[0] == "@ATTRIBUTE":
                        attributes.append(row_splitted[1])
                    
                    # If we found that @data started
                    if row[0] == "@data":
                        dataBegan = True
                        continue
            
            # Reading only payload of the file
            if dataBegan:
                if len(row) > 1:
                    # Appending row into data array
                    data.append(row)
                    line_num += 1

                    # Limiting the number of rows to read
                    if line_num > limit and limit > 0:
                        break
        
    return (data, attributes)

In [4]:
# Processing WEKA file
(data, attr) = loadWEKA('youTubeLocationIDWeka.csv', limit=0)

#### Thats how weka file looks like now. Array of Arrays:

In [5]:
data[:2]

[['1892013124614', '', '', '', '', 'East Contoy Island Mexico', '2c'],
 ['1892013125327', '', '', '', '', 'East Contoy Island Mexico', '2c']]

#### Thats the column names loadWEKA() extracted:

In [13]:
attr

['encounter', 'video', 'title', 'tags', 'description', 'location', 'class']

### Matching YT video titles with their location
Naive way looping through the title and description word by word, trying to match the location:

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [8]:
def matchLocation(data, props, description=False, output=False):
    count = 0
    identified = 0
    spacy_count = 0
    
    for item in data:
        # Working with YT videos only
        if item[1]:
            found = False
            
            # Going through title word by word
            title_splitted = item[2].split()
            for word in title_splitted:
                try:
                    if (props[word.lower()]):
                        if output:
                            print("Found", word, "in title", item[2], ", id", item[0])
                        identified += 1
                        found = True
                        break # Breaking the loop, going to the next video
                except:
                    pass
            
            if not found and description:
                # Going through description word by word
                descr_splitted = item[4].split()
                for word in descr_splitted:
                    try:
                        if (props[word.lower()]):
                            if output:
                                print("Found", word, "in description", item[4], ", id", item[0])
                            identified += 1
                            break # Breaking the loop, going to the next video
                    except:
                        for ent in nlp(item[2].replace("'","")).ents:
                            # print(ent.text, ent.start_char, ent.label_) #Print if needed 
                            if ent.label_ == "GPE":
                                spacy_count += 1
                                print("\n", item)
                                print(ent.text)
                        pass
            
            count += 1

    print("Processed ", count, " videos")
    print("Identified ", identified, " locations")
    print("Identified by spacy ", spacy_count, " locations")
    
matchLocation(data, props, description=True, output=False)


 ['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriWhale sharkSqualo balenaSharksSqualiMurenaMorayStingrayTrigoneAquila di mareRays fishPesce chirurgoSurgeonfishLion fishpesce leonepesce scorpioneTartarugheturtlesmantasnorkelinggopro hero 7 blackgopro hero 4 black', 'In this video you can see the Whale shark Sharks Manta Stingray Surgeonfish Lion fish Turtles Rays fish and much moreFilmed with GoPro Hero 7 Black and GoPro Hero 4 BlackIn questo video potrete vedere lo squalo balena diversi squali tartarughe pesci chirurgo mante trigone aquile di mare e molto altroFilmato con GoPro Hero 7 Black e GoPro hero 4 Black', 'Ari Atol Maldive', 'Maldives']
Thudufushi

 ['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriW


 ['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriWhale sharkSqualo balenaSharksSqualiMurenaMorayStingrayTrigoneAquila di mareRays fishPesce chirurgoSurgeonfishLion fishpesce leonepesce scorpioneTartarugheturtlesmantasnorkelinggopro hero 7 blackgopro hero 4 black', 'In this video you can see the Whale shark Sharks Manta Stingray Surgeonfish Lion fish Turtles Rays fish and much moreFilmed with GoPro Hero 7 Black and GoPro Hero 4 BlackIn questo video potrete vedere lo squalo balena diversi squali tartarughe pesci chirurgo mante trigone aquile di mare e molto altroFilmato con GoPro Hero 7 Black e GoPro hero 4 Black', 'Ari Atol Maldive', 'Maldives']
Thudufushi

 ['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriW


 ['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriWhale sharkSqualo balenaSharksSqualiMurenaMorayStingrayTrigoneAquila di mareRays fishPesce chirurgoSurgeonfishLion fishpesce leonepesce scorpioneTartarugheturtlesmantasnorkelinggopro hero 7 blackgopro hero 4 black', 'In this video you can see the Whale shark Sharks Manta Stingray Surgeonfish Lion fish Turtles Rays fish and much moreFilmed with GoPro Hero 7 Black and GoPro Hero 4 BlackIn questo video potrete vedere lo squalo balena diversi squali tartarughe pesci chirurgo mante trigone aquile di mare e molto altroFilmato con GoPro Hero 7 Black e GoPro hero 4 Black', 'Ari Atol Maldive', 'Maldives']
Thudufushi

 ['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriW


 ['6a27654d-a2cc-4c37-a4c8-d152663974d9', 'https://www.youtube.com/watch?v=KMYGLQh-ujI', 'Whale Shark  Walhaie  Mexico 2018  Viva Wyndham Maya', 'whale sharkwalhaie mexikowalhai tauchengopro sharkgopromexicoviva wyndham mayaholidayurlaub in mexicotauchen in mexikomexiko tauchenwalhai schnorcheln in mexikoschnorcheln mexiko cancunsnorkelcancun mexicocancunplaya del carmen mexicoplaya del carmenplaya del carmen 2018justyna baloniakmichael schnfelder', 'Walhaie sind die grten Fische der Erde die Aufnahmen sind in unserem Urlaub 2018 in Mexico entstanden', '', '2c']
Mexico

 ['6a27654d-a2cc-4c37-a4c8-d152663974d9', 'https://www.youtube.com/watch?v=KMYGLQh-ujI', 'Whale Shark  Walhaie  Mexico 2018  Viva Wyndham Maya', 'whale sharkwalhaie mexikowalhai tauchengopro sharkgopromexicoviva wyndham mayaholidayurlaub in mexicotauchen in mexikomexiko tauchenwalhai schnorcheln in mexikoschnorcheln mexiko cancunsnorkelcancun mexicocancunplaya del carmen mexicoplaya del carmenplaya del carmen 2018justy


 ['41391d5d-4760-4f0f-9f71-28032f25625a', 'https://www.youtube.com/watch?v=KMYGLQh-ujI', 'Whale Shark  Walhaie  Mexico 2018  Viva Wyndham Maya', 'whale sharkwalhaie mexikowalhai tauchengopro sharkgopromexicoviva wyndham mayaholidayurlaub in mexicotauchen in mexikomexiko tauchenwalhai schnorcheln in mexikoschnorcheln mexiko cancunsnorkelcancun mexicocancunplaya del carmen mexicoplaya del carmenplaya del carmen 2018justyna baloniakmichael schnfelder', 'Walhaie sind die grten Fische der Erde die Aufnahmen sind in unserem Urlaub 2018 in Mexico entstanden', '', '2c']
Mexico

 ['41391d5d-4760-4f0f-9f71-28032f25625a', 'https://www.youtube.com/watch?v=KMYGLQh-ujI', 'Whale Shark  Walhaie  Mexico 2018  Viva Wyndham Maya', 'whale sharkwalhaie mexikowalhai tauchengopro sharkgopromexicoviva wyndham mayaholidayurlaub in mexicotauchen in mexikomexiko tauchenwalhai schnorcheln in mexikoschnorcheln mexiko cancunsnorkelcancun mexicocancunplaya del carmen mexicoplaya del carmenplaya del carmen 2018justy

...


 ['89ceec5c-20da-434e-b198-3be67e1ead4f', 'https://www.youtube.com/watch?v=kOHLUw4jaS4', 'A whale shark cruises in very shallow water in La Paz Mexico', 'whale sharkendangered speciesiucnwhale shark conservationla paz bcsla paz mexicothings to do in la paz mexicowhale shark diarieswhale sharks mexicowhale sharks la pazwhale sharks baja', 'A six metre female whale shark cruising along in just 2  25 metres depth of water close to the shore of the Mogote sandspit in La Paz Baja California Sur Mexico The water is so shallow that the shark is dragging the bottom of her tail in the sandA second whale shark passes in the opposite direction towards the end of the clipFor details of our whale trips and whale shark conservation internships visit ', 'La Paz Baja California Sur Mexico', 'Mexico-BLP']
La Paz Mexico

 ['89ceec5c-20da-434e-b198-3be67e1ead4f', 'https://www.youtube.com/watch?v=kOHLUw4jaS4', 'A whale shark cruises in very shallow water in La Paz Mexico', 'whale sharkendangered speciesi


 ['c8bbd853-c826-42ef-81d6-63dae6f6c017', 'https://www.youtube.com/watch?v=Wr0y69uodus', 'Pescador ayuda a tiburn ballena', 'ballenatiburontiburon ballenatiburn ballenavideomartiburnnoticiasplayapeztiburonestiburon blancoballena azulayudashark animalanimalesmexicopecesanimalbuceorescatebarcooceano', 'Parece ser que tenia algo ah hacindole dao menuda herida se le escapan los peces por el agujero', '', 'null']
Pescador

 ['c8bbd853-c826-42ef-81d6-63dae6f6c017', 'https://www.youtube.com/watch?v=Wr0y69uodus', 'Pescador ayuda a tiburn ballena', 'ballenatiburontiburon ballenatiburn ballenavideomartiburnnoticiasplayapeztiburonestiburon blancoballena azulayudashark animalanimalesmexicopecesanimalbuceorescatebarcooceano', 'Parece ser que tenia algo ah hacindole dao menuda herida se le escapan los peces por el agujero', '', 'null']
Pescador

 ['c8bbd853-c826-42ef-81d6-63dae6f6c017', 'https://www.youtube.com/watch?v=Wr0y69uodus', 'Pescador ayuda a tiburn ballena', 'ballenatiburontiburon ballenat


 ['31a756b9-144d-4855-9982-493de2a98bd8', 'https://www.youtube.com/watch?v=Wr0y69uodus', 'Pescador ayuda a tiburn ballena', 'ballenatiburontiburon ballenatiburn ballenavideomartiburnnoticiasplayapeztiburonestiburon blancoballena azulayudashark animalanimalesmexicopecesanimalbuceorescatebarcooceano', 'Parece ser que tenia algo ah hacindole dao menuda herida se le escapan los peces por el agujero', '', 'null']
Pescador

 ['31a756b9-144d-4855-9982-493de2a98bd8', 'https://www.youtube.com/watch?v=Wr0y69uodus', 'Pescador ayuda a tiburn ballena', 'ballenatiburontiburon ballenatiburn ballenavideomartiburnnoticiasplayapeztiburonestiburon blancoballena azulayudashark animalanimalesmexicopecesanimalbuceorescatebarcooceano', 'Parece ser que tenia algo ah hacindole dao menuda herida se le escapan los peces por el agujero', '', 'null']
Pescador

 ['31a756b9-144d-4855-9982-493de2a98bd8', 'https://www.youtube.com/watch?v=Wr0y69uodus', 'Pescador ayuda a tiburn ballena', 'ballenatiburontiburon ballenat


 ['88391b48-7a3d-48ad-90ac-3da986b120ba', 'https://www.youtube.com/watch?v=aCh_cVblGYE', 'Catamaran cruise workers cut whale shark free from net off Kauai coast', 'defaultlocal news', 'Excitement turned to concern when they learned the whale shark was entangled in fishing net', 'Kauai Hawaii', 'Hawaii']
Catamaran

 ['88391b48-7a3d-48ad-90ac-3da986b120ba', 'https://www.youtube.com/watch?v=aCh_cVblGYE', 'Catamaran cruise workers cut whale shark free from net off Kauai coast', 'defaultlocal news', 'Excitement turned to concern when they learned the whale shark was entangled in fishing net', 'Kauai Hawaii', 'Hawaii']
Catamaran

 ['88391b48-7a3d-48ad-90ac-3da986b120ba', 'https://www.youtube.com/watch?v=aCh_cVblGYE', 'Catamaran cruise workers cut whale shark free from net off Kauai coast', 'defaultlocal news', 'Excitement turned to concern when they learned the whale shark was entangled in fishing net', 'Kauai Hawaii', 'Hawaii']
Catamaran

 ['88391b48-7a3d-48ad-90ac-3da986b120ba', 'https://

Processed  2580  videos
Identified  1524  locations
Identified by spacy  1648  locations
